<a href="https://colab.research.google.com/github/Josiah-tan/ez_life/blob/main/ez_life/jt_property.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JTProperty Full Implementation 
- Code below shows full implementation of the JTproperty class
  - Skip this section if you don't really want to look at this implementation
    - Open table of contents, and go to Basic Property Demo to start
  - If curious, feel free to read thru the code


## Importing Dependencies

In [1]:
import copy
import functools
import sys
import threading

## Graph Datastructure
- Implementation for graph dependencies

In [2]:
class Node:
  def __init__(self, data):
    self.data = data
    self.was_visited = False
    self._edges = set()

  def addEdge(self, edges):
    assert isinstance(edges, set)
    self._edges.update(edges) 


class Graph:
  def __init__(self, cls_name):
    self.cls_name = cls_name
    self.data2node = {}

  def data2Node(self, data):
    node = self.data2node.get(data, None)
    if node is None:
      node = Node(data)
      self.data2node[data] = node
    return node

  def add(self, out, into):
    out = out if isinstance(out, list) else {out}
    into = into if isinstance(into, list) else {into}

    outNodes = {self.data2Node(o) for o in out}
    intoNodes = {self.data2Node(i) for i in into}

    for outNode in outNodes:
      outNode.addEdge(intoNodes)
  
  def disp(self):
    print("displaying graph")
    for data, node in self.data2node.items():
      print(f"{data} points to {set(n.data for n in node._edges)}")

  
  def resetDepDFS(self, obj, protected_name):
    """
    Runs a DFS alrgorithm on the graph datastructure to reset all downstream dependencies to None
    """
    node = self.data2Node(protected_name)

    def recursiveReset(node):
      if (not isinstance(node, Node)) or node.was_visited:
        return
      node.was_visited = True
      if (node.data in dir(obj)) and getattr(obj, node.data) is not None: # added this here so that recursion stops when the attribute is already None such that downstream dependencies are not reset since they are assumed to be also None, or preset to some value 
        setattr(obj, node.data, None)
        [recursiveReset(n) for n in node._edges]
  
    recursiveReset(node)

    for n in self.data2node.values():
      n.was_visited = False


In [3]:
class ClsGraph(dict):
  def resetDepDFS(self, cls, obj, protected_name):
    """
    Runs a DFS alrgorithm on the graph datastructure to reset all downstream dependencies to None
    parameters
      -- obj: the class object that we are dealing with
      -- cls = type(obj)
      -- protected_name: the name prefixed with an underscore
    """

    """ find node of protected_name """

    """
    print("resetDepDFS start")
    for parent in cls.__mro__[:-1]:
      for node in self[parent].data2node.values():
        print(f"parent {parent} node.data {node.data} node {node} node._edges {node._edges}")
    print("resetDepDFS end")

    """

    #for cls, graph in self.items():
    node = None
    for parent in cls.__mro__[:-1]:
      node = self[parent].data2node.get(protected_name, None)
      if node is not None:
          break

    if node is None:
      return

    def recursiveReset(node):
      if (not isinstance(node, Node)) or node.was_visited:
        return
      node.was_visited = True
      if (node.data in dir(obj)) and getattr(obj, node.data) is not None: # added this here so that recursion stops when the attribute is already None such that downstream dependencies are not reset since they are assumed to be also None, or preset to some value 
        setattr(obj, node.data, None)
        [recursiveReset(n) for n in node._edges]
  
    recursiveReset(node)

    for parent in cls.__mro__[:-1]:
      for n in self[parent].data2node.values():
        n.was_visited = False

In [4]:
if __name__ == "__main__":
  cls_dict = ClsGraph()
  cls_dict['1'] = 4
  print(type(cls_dict))

<class '__main__.ClsGraph'>


## DefaultSetter
- DefaultSetter is a class that sets a default identity setter when setter = "default" is called via the JTProperty class

In [5]:
class DefaultSetter:
    def __init__(self, setter):
      self.setter = setter
    def __call__(self, _func):
      if str(self.setter).lower() in "default":
        _func = _func.setter(self.defaultSetter)
      return _func
    
    @staticmethod
    def defaultSetter(obj, var):
      return var


## EzProperty
  - EzProperty function returns an object that inherits from the property class
    - Currently EzProperty changes the setter function such that the return value is the value set by the setter

In [6]:
def EzProperty(JTProperty_obj):
  class ClsWrapper(property):
    def __init__(self, *args, **kwargs):
      return super().__init__(*args, **kwargs)
    
    def setter_preprocess(self, _func):
      """
      Performs preprocessing on the self._func decorated by @func.setter
        - resets all downstream graph dependencies
        - sets return value of _func to protected name of _func
      """
      def wrapper(obj, val):
        #JTProperty_obj.firstBeforeS_Get()
        JTProperty_obj.joinClsThreads()

        #cls_old = JTProperty_obj.cls_name2cls[JTProperty_obj.cls_name]
        #print(f'cls_old {cls_old}')
        cls = type(obj)
        #print(f"cls {cls}")
        JTProperty_obj.cls2graph[cls].resetDepDFS(cls, obj, JTProperty_obj.protected_name)

        #JTProperty_obj.cls_name2graph[JTProperty_obj.cls_name].resetDepDFS(obj, JTProperty_obj.protected_name)
        setattr(obj, JTProperty_obj.protected_name, _func(obj, val))
        #print(getattr(obj, protected_name))
      return wrapper
  
    def setter(self, _func):
      """
      calls setter_preprocess wrapper to alter behaviour of _func
      """
      return super().setter(self.setter_preprocess(_func))
  return ClsWrapper

## JTProperty
- The Main class decorator

In [7]:

class JTProperty:

  # dicts for clsWasDeclared multithreading
  cls_name2cls = {}
  cls_name2thread = {}
  cls_name2active_t = {} # stores currently active threads

  # contains scaffolds of cls graphs
  cls_name2graph = {}
  # contains actual cls graphs
  cls2graph = {}

  # maps protected variable names to a JTProperty object
  #protected_name2self = {}

  # maps the name of the classes to a list of protected variables which were defined in a class
  cls_name2protected_names = {}

  def __init__(self, setter = False, deps = None):
    # Tri state "setter": True, Default, False
    self.setter = setter
    self.deps = self.preprocessDeps(deps)
  
  def preprocessDeps(self, deps):
    """
    converts all deps to protected string variables
    """
    if deps is None:
      return None
    elif not isinstance(deps, (list, set)):
      deps = [deps]
    #check if all dependencies are either a string (or a EzProperty instance <- not implemented)
    assert all(isinstance(dep, (str)) for dep in deps)
    return [f"_{dep}" for dep in deps]

  def getVar(self, obj):
    # if self._name is not available atm or it is set to None
    if (self.protected_name not in dir(obj)) or (getattr(obj, self.protected_name) is None):
      if self.setter == False:
        setattr(obj, self.protected_name, self._func(obj)) 
      else:
        # call setter method obj.name with the return value of the property function, this effectively sets obj._name
        setattr(obj, self.public_name, self._func(obj)) 
    return getattr(obj, self.protected_name)

  def initDepGraph(self):
    cls_graph = self.cls_name2graph.get(self.cls_name, None)
    if cls_graph is None:
      self.cls_name2graph[self.cls_name] = Graph(self.cls_name)

    if self.deps is not None:
      self.cls_name2graph[self.cls_name].add(out = self.deps, into = self.protected_name)
      #self.cls_name2graph[cls_name].disp()
    else: # important for inheritence
      self.cls_name2graph[self.cls_name].add(out = [], into = self.protected_name)

  def __call__(self, _func):
    self._func = _func
    self.public_name = _func.__name__
    self.protected_name = f"_{self.public_name}"

    #self.protected_name2self[self] = self.protected_name # useful for inheritence node connection later on

    self.cls_name = _func.__qualname__.rsplit('.', 1)[0]
    #cls = inspect._findclass(_func) <- big annoying problem: can't get cls from _func within this __call__ method, cls is not a global variable yet

    #self.protected_name2cls_name[self.protected_name] = self.cls_name
    if self.cls_name2protected_names.get(self.cls_name, None) is None:
      self.cls_name2protected_names[self.cls_name] = [self.protected_name]
    else:
      self.cls_name2protected_names[self.cls_name].append(self.protected_name) 

    self.initDepGraph()

    # the getter method here
    @DefaultSetter(self.setter)
    @EzProperty(self)
    @functools.wraps(_func)
    def wrapper(obj):
      #might need something here to connect nodes with inherited classes
      self.joinClsThreads()
      #self.firstBeforeS_Get()
      return self.getVar(obj)

    # perform tasks after cls is declared (run this last to reduce busy waiting load)
    self.clsWasDeclared()
    return wrapper

  def clsWasDeclared(self):
    def cls_name2Cls():
      # Need someone to help me make a better listener than this plz
      while self.cls_name not in dir(sys.modules.get(self._func.__module__)):
        pass

      # essentially the same thing as in inspect.py -> _findclass() 
      cls = sys.modules.get(self._func.__module__)
      for name in self.cls_name.split('.'):
        cls = getattr(cls, name) 

      self.cls_name2cls[self.cls_name] = cls #getattr(sys.modules.get(self._func.__module__), self.cls_name)

      # do tasks here after cls is declared
      self.createDepGraph(cls)
    
    # we only run this block once for the first decorated method in the class
    if self.cls_name2thread.get(self.cls_name, None) is None:
      #NOTE! Might need to join pre-existing threads somewhere in the code, maybe
      self.cls_name2thread[self.cls_name] = threading.Thread(target = cls_name2Cls)
      self.cls_name2thread[self.cls_name].start()

      self.cls_name2active_t[self.cls_name] = self.cls_name2thread[self.cls_name] # to denote that its running

  def createDepGraph(self, cls):
    """ ensuring that all parent threads are complete"""
    for parent in cls.__mro__[-2:0:-1]: # exclude beginning and end, and reverse the list
      parent_name = parent.__qualname__
      #print(f'parent_name {parent_name}')
      parent_t = self.cls_name2active_t.pop(parent_name, None)
      if parent_t is not None:
        #print(f'parent_joined {parent_name}')
        parent_t.join()
    
    """cls2graph initialisation and copying class parent graphs"""
    self.cls2graph[cls] = ClsGraph({cls : self.cls_name2graph[self.cls_name]}) #ClsGraph inherits from dict, has method resetDepGraph

    for parent in cls.__bases__: # only use parents that are directly inherited
      if parent is not object:
        self.cls2graph[cls].update(copy.deepcopy(self.cls2graph[parent])) # make deep copies of the parent graphs to prevent effecting the parent graph

    """connecting nodes from parent classes"""
    cls_graph = self.cls2graph[cls]
    prot_names2del = []
    for data, node in cls_graph[cls].data2node.items(): # data is the protected decorated _func name
      for parent in cls.__mro__[:-1]: # we want to go in order of how Python searches through the inherited classes excluding final object class
        parent_name = parent.__qualname__
        if data in self.cls_name2protected_names[parent_name]: # check if the protected variable was defined in this parent class
          if parent is not cls: # don't need to reconnect nodes when the protected variable is inside cls
            parent_node = cls_graph[parent].data2node[data]
            #print(f"node._edges {node._edges}")
            parent_node.addEdge(node._edges)
            #print(f"parent_node._edges {parent_node._edges}")
            prot_names2del.append(data)
          break # make sure to get out of the MRO loop as soon as u find the variable (cause that's just how MRO works)

    # deleting uneccessary nodes
    for protected_name in prot_names2del:
        del cls_graph[cls].data2node[protected_name]
    
  def joinClsThreads(self):
    while len(self.cls_name2active_t) != 0:
      cls_name, active_t = self.cls_name2active_t.popitem()
      active_t.join()

    #for cls_name, thread in self.cls_name2thread.items():
      #thread.join()
    """
    thread = self.cls_name2thread.get(self.cls_name, None)
    if thread is None:
      pass
    else:
      thread.join()
    """

## A Small Helper Function
- Helps with debug testing

In [8]:
if __name__ == '__main__':
  def print_assert(p, a = None):
    print(p)
    if a is not None:
      assert p.__str__() == a

## Hierarchical Graph Dependencies
- There is also support for graphs that use inherited methods that are decorated with JTProperty

In [9]:
class ParentGraphDemo:
  @JTProperty(setter = "Default")
  def a(self):
    return 'a'

class ChildGraphDemo(ParentGraphDemo):
  @JTProperty(setter = "Default", deps = 'a')
  def b(self):
    return self.a + '->b'
  
  @JTProperty(setter = "Default")
  def c(self):
    return 'c'

  @JTProperty(setter = "Default", deps = ['c', 'b'])
  def d(self):
    return self.b + '->d' + ' and ' + self.c + '->d'

In [10]:
print(JTProperty.cls2graph[ChildGraphDemo][ParentGraphDemo])
print(JTProperty.cls2graph[ChildGraphDemo][ParentGraphDemo].data2node)

print(JTProperty.cls2graph[ChildGraphDemo][ParentGraphDemo].data2node['_a']._edges)

{'_a': <__main__.Node object at 0x7f56f2a00c50>}
{<__main__.Node object at 0x7f56f2a008d0>}


In [11]:
if __name__ == '__main__':
  graph_demo = ChildGraphDemo()
  print_assert(graph_demo.d, 'a->b->d and c->d')
  graph_demo.a = 'A'
  print_assert(graph_demo.d, 'A->b->d and c->d')
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_a points to {'_b'}
  #_b points to {'_d'}
  #_c points to {'_d'}
  #_d points to set()

a->b->d and c->d
A->b->d and c->d
displaying graph
_b points to {'_d'}
_c points to {'_d'}
_d points to set()


In [12]:
if __name__ == '__main__':
  graph_demo = ChildGraphDemo()
  graph_demo.d = 'a->b->d and c->d'
  graph_demo.a = 'A'
  print_assert(graph_demo._d, 'a->b->d and c->d')
  print_assert(graph_demo.d, 'a->b->d and c->d')
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_a points to {'_b'}
  #_b points to {'_d'}
  #_c points to {'_d'}
  #_d points to set()

a->b->d and c->d
a->b->d and c->d
displaying graph
_b points to {'_d'}
_c points to {'_d'}
_d points to set()


### Deep Inheritence

In [13]:
class ParentAGraphDemo:
  @JTProperty(setter = "Default")
  def a(self):
    return 'a'

class ParentBGraphDemo(ParentAGraphDemo):
  @JTProperty(setter = "Default", deps = 'a')
  def b(self):
    return self.a + '->b'

class ParentCGraphDemo(ParentBGraphDemo):
  @JTProperty(setter = "Default")
  def c(self):
    return 'c'

class DGraphDemo(ParentCGraphDemo):
  @JTProperty(setter = "Default", deps = ['c', 'b'])
  def d(self):
    return self.b + '->d' + ' and ' + self.c + '->d'

In [14]:
if __name__ == '__main__':
  graph_demo = DGraphDemo()
  print_assert(graph_demo.d, 'a->b->d and c->d')
  graph_demo.a = 'A'
  print_assert(graph_demo.d, 'A->b->d and c->d')
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_a points to {'_b'}
  #_b points to {'_d'}
  #_c points to {'_d'}
  #_d points to set()

a->b->d and c->d
A->b->d and c->d
displaying graph
_d points to set()


In [15]:
if __name__ == '__main__':
  graph_demo = DGraphDemo()
  graph_demo.d = 'a->b->d and c->d'
  graph_demo.a = 'A'
  print_assert(graph_demo._d, 'a->b->d and c->d')
  print_assert(graph_demo.d, 'a->b->d and c->d')
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_a points to {'_b'}
  #_b points to {'_d'}
  #_c points to {'_d'}
  #_d points to set()

a->b->d and c->d
a->b->d and c->d
displaying graph
_d points to set()


# Basic Property Demo
- prop3 is dependent upon the values of prop2 and prop 1 as shown below

In [16]:
class PropDemo:
  def __init__(self):
    self._prop1 = None
    self._prop2 = None
    self._prop3 = None
  
  @property
  def prop1(self):
    if self._prop1 is None:
      self._prop1 = self.get_prop1()
    return self._prop1

  @property
  def prop2(self):
    if self._prop2 is None:
      self._prop2 = self.get_prop2()
    return self._prop2

  @property
  def prop3(self):
    if self._prop3 is None:
      self._prop3 = self.get_prop3()
    return self._prop3
 
  def get_prop1(self):
    return 1
  def get_prop2(self):
    return self.prop1 + 1
  def get_prop3(self):
    return self.prop2 + 1
  


In [17]:
if __name__ == '__main__':
  prop_dem = PropDemo()
  print_assert(prop_dem.prop3, '3')

3


- The @JTProperty decorator uses less lines of code then the @property decorator, but achieves the same result

In [18]:
class JTPropDemo:
  def __init__(self):
    pass
  
  @JTProperty()
  def prop1(self):
    return 1

  @JTProperty()
  def prop2(self):
    return self.prop1 + 1

  @JTProperty()
  def prop3(self):
    return self.prop2 + 1
  

In [19]:
if __name__ == '__main__':
  prop_dem = JTPropDemo()
  print_assert(prop_dem.prop3, '3')


3


# Setter methods
- Consider a class that uses getter and setter methods as shown below:

In [20]:
class SetAndGet:
  def __init__(self, r = 1):
    # initialise the protected variable
    self._radius = None

    # calls the @radius.setter method
    self.radius = r
  @property
  def radius(self):
    if self._radius is None:
      self.radius = 2
    return self._radius
  @radius.setter
  def radius(self, r):
    if r <= 0:
      raise ValueError("radius should be greater than 0")
    self._radius = r

- In the test below, contextlib silences the ValueError that occurs with setting the radius to -5

In [21]:
if __name__ == '__main__':
  import contextlib
  setandget = SetAndGet()
  #print(setandget.radius)
  print_assert(setandget.radius, '1')

  setandget.radius = 5
  #print(setandget.radius)
  print_assert(setandget.radius, '5')

  setandget.radius = 3
  #print(setandget.radius)
  print_assert(setandget.radius, '3')

  with contextlib.suppress(ValueError):
    setandget.radius = -5
  #print(setandget.radius)
  print_assert(setandget.radius, '3')

1
5
3
3


- JTProperty() and .setter reduce abstraction involving usage of hidden "protected variables"
  - setter = True should be set when @radius.setter is used

In [22]:
class JTSetAndGet:
  def __init__(self, r = 1):
    self.radius = r
  @JTProperty(setter = True)
  def radius(self):
    return 2

  @radius.setter
  def radius(self, r):
    if r <= 0:
      raise ValueError("radius should be greater than 0")
    return r


In [23]:
if __name__ == '__main__':
  import contextlib
  setandget = SetAndGet()
  print_assert(setandget.radius, '1')

  setandget.radius = 5
  print_assert(setandget.radius, '5')

  setandget.radius = 3
  print_assert(setandget.radius, '3')

  with contextlib.suppress(ValueError):
    setandget.radius = -5
  print_assert(setandget.radius, '3')

1
5
3
3


# Automatic Setter 
- The JTProperty decorator allows for automatically generated setters
- The code below demonstrates the explicit version, followed by the implicit version
  - The explicit version has the same functionality as the implicit version

In [24]:
# writing the setter explicitly
class SetterDemo:
  @JTProperty(setter=True)
  def prop(self):
    return 1
  
  @prop.setter
  def prop(self, val):
    return val

In [25]:
if __name__ == "__main__":
  # test setter before getter
  setter_demo = SetterDemo()
  setter_demo.prop = 2
  print_assert(setter_demo.prop, '2')

2


In [26]:
if __name__ == "__main__":
  # test getter before setter
  setter_demo = SetterDemo()
  print_assert(setter_demo.prop, '1')

1


In [27]:
# writing the setter implicitly
class AutoSetterDemo:
  @JTProperty(setter="Default")
  def prop(self):
    return 1


In [28]:
if __name__ == "__main__":
  # test setter before getter
  auto_setter_demo = AutoSetterDemo()
  auto_setter_demo.prop = 2
  print_assert(auto_setter_demo.prop, '2')

2


In [29]:
if __name__ == "__main__":
  # test getter before setter
  auto_setter_demo = AutoSetterDemo()
  print_assert(auto_setter_demo.prop, '1')

1


# Graph Dependencies 



<img src="https://drive.google.com/uc?export=view&id=149jAbjHU9BWt_W5pRYqn0bQbLFVz-w8y" width="300" align="right"> 

- Consider a graph as shown to the right:
    1. if a updates, then b updates, then d updates
    1. if b updates, then d updates
    1. if c updates, then d updates 
    1. if d updates, nothin happens bro 

## More Implementational Details

- Note a few important features in regards to the implementation:
  - If a node variable is None, graph traversal stemming from that node is stopped
    - Consider case 1 as an example:
      - If b is None and a is updated, then d does not get updated because b stops the traversal
      - When a variable is set to some initial value, there is no dependency graph traversal stemming from that node
  - Whenever upstream variables "update" they are actually set to None
    - This reduces memory load and uneccessary computation when not needed
    - These variables are computated only when called




## Examples 

- An example use case is shown below: 


In [30]:
class GraphDemo:
  @JTProperty(setter = "Default")
  def a(self):
    return 'a'

  @JTProperty(setter = "Default", deps = 'a')
  def b(self):
    return self.a + '->b'
  
  @JTProperty(setter = "Default")
  def c(self):
    return 'c'

  @JTProperty(setter = "Default", deps = ['c', 'b'])
  def d(self):
    return self.b + '->d' + ' and ' + self.c + '->d'

In [31]:
if __name__ == '__main__':
  graph_demo = GraphDemo()
  print_assert(graph_demo.d, 'a->b->d and c->d')
  graph_demo.a = 'A'
  print_assert(graph_demo.d, 'A->b->d and c->d')
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_a points to {'_b'}
  #_b points to {'_d'}
  #_c points to {'_d'}
  #_d points to set()

a->b->d and c->d
A->b->d and c->d
displaying graph
_a points to {'_b'}
_b points to {'_d'}
_c points to {'_d'}
_d points to set()


In [32]:
if __name__ == '__main__':
  graph_demo = GraphDemo()
  graph_demo.d = 'a->b->d and c->d'
  graph_demo.a = 'A'
  print_assert(graph_demo._d, 'a->b->d and c->d')
  print_assert(graph_demo.d, 'a->b->d and c->d')
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_a points to {'_b'}
  #_b points to {'_d'}
  #_c points to {'_d'}
  #_d points to set()

a->b->d and c->d
a->b->d and c->d
displaying graph
_a points to {'_b'}
_b points to {'_d'}
_c points to {'_d'}
_d points to set()


## Circular Graph Dependencies
<img src="https://drive.google.com/uc?export=view&id=1U1QLwQMqZxv77M0GU1UAMNt4kYO4JNqO" width="300" align = "right"> 
- Consider the graph to the right:
  1. If A updates, B updates, then C and D
  1. If B updates, A updates, then C and D
  1. If C updates, D updates, then A and B
  1. If D updates, A updates, then B and C
- Circular Graph Dependencies are like normal dependencies, however there can be specific problems when using them such as infinite recursive calls, for example:
  - Say that C is set to some value ___c___
  - When self.b is called, it calls:
    - self.a, which calls
      - self.b and self.d
        - self.d calls self.c which returns ___c___
        - self.b calls self.a
  - We can see there is a recursive problem here as self.a calls self.b and vice versa since these values have not been preset to some initial value, however there is no problem with self.d because it depends only on self.c which is set to ___c___
  - Now, consider a set to some value ___a___
    - When self.d is called, it calls
      - self.c which calls
        - self.b which calls
          - self.a which resolves to ___a___
    - Here, the value of self.d evaluates to some value, hence there are no problems






In [33]:
class GraphDemo2:
  @JTProperty(setter = "Default", deps = ['b', 'd'])
  def a(self):
    return self.b + '->a and ' + self.d + '->a'

  @JTProperty(setter = "Default", deps = 'a')
  def b(self):
    return self.a + '->b'
  
  @JTProperty(setter = "Default", deps = 'b')
  def c(self):
    return self.b + '->c'

  @JTProperty(setter = "Default", deps = ['c'])
  def d(self):
    return self.c + '->d'

In [34]:
if __name__ == '__main__':
  graph_demo = GraphDemo2()
  JTProperty.cls_name2graph[type(graph_demo).__name__].disp()
  #displaying graph
  #_b points to {'_c', '_a'}
  #_a points to {'_b'}
  #_c points to {'_d'}
  #_d points to {'_a'}

  # tests if setter for .b resets ._a accidentally
  graph_demo = GraphDemo2()
  graph_demo.a = 'a'
  print_assert(graph_demo.a, 'a')
  print_assert(graph_demo.b, 'a->b')
  print_assert(graph_demo._a, 'a')
  
  graph_demo = GraphDemo2()
  print("graph_demo.a = 'a':")
  graph_demo.a = 'a'
  print_assert(graph_demo.b, 'a->b')
  print_assert(graph_demo.c, 'a->b->c')
  print_assert(graph_demo.d, 'a->b->c->d')
  
  graph_demo = GraphDemo2()
  print("graph_demo.b = 'b':")
  graph_demo.b = 'b'
  print_assert(graph_demo.d, 'b->c->d')
  print_assert(graph_demo._b, 'b')
  print_assert(graph_demo.a, 'b->a and b->c->d->a')
  print_assert(graph_demo.c, 'b->c')
  
  # Causes Recursion problems (intentional) <- a and b must be preset since they are dependent on each other
  #graph_demo = GraphDemo2()
  #print("graph_demo.c = 'c':")
  #graph_demo.c = 'c'
  #print(graph_demo.a)
  #print(graph_demo.b)
  #print(graph_demo.d)
  
  # Causes Recursion problems (intentional) <- a and b must be preset since they are dependent on each other
  #graph_demo = GraphDemo2()
  #graph_demo.d = 'd'
  #print(graph_demo.a)
  #print(graph_demo.b)
  #print(graph_demo.c)

displaying graph
_b points to {'_c', '_a'}
_d points to {'_a'}
_a points to {'_b'}
_c points to {'_d'}
a
a->b
a
graph_demo.a = 'a':
a->b
a->b->c
a->b->c->d
graph_demo.b = 'b':
b->c->d
b
b->a and b->c->d->a
b->c


# Areas Of Improvement
  - Introduce setter detection to reduce uneccessary setter = True kwargs
  - Dependency graphs + inheritence using graph clusters
  - Multithreaded dependency dependency computation


# Debates
- Whenever a node is updated should all downstream dependencies be set to None?
- Consider a graph that looks like this:
  - a->b->c and c->a
  - consider setting c = ___c___ and then calling a
    - a depends on c, thus a is set to some value
      - However the act of setting a causes c to reset to None as via the nature of the DFS graph traversal
    - Therefore, essentially only one variable can exist at a time which doesn't make sense within a circular dependency setting


In [35]:
# checking for any residual threads lyin about
if __name__ == "__main__":
  print(threading.enumerate())

[<_MainThread(MainThread, started 140011891709824)>, <Thread(Thread-2, started daemon 140011495732992)>, <Heartbeat(Thread-3, started daemon 140011487340288)>, <ParentPollerUnix(Thread-1, started daemon 140011422996224)>]
